# Fine Tuning

In [17]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=7704bb2d7f5eb3589e31b37133732a2f4a836ed894d906f1fc890c932155e1e7
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score




```
# Questo è formattato come codice
```

## Import libraries

In [15]:
from tqdm import tqdm
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_metric
from datasets import load_dataset
import pandas as pd
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import evaluate
import pandas as pd

## dataset

In [4]:
dataset = load_dataset("json", data_files={'train':'train.json', 'test': 'test.json', 'validation':'validate.json'})
split_lengths = [len(dataset[split])for split in dataset]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\nPlain_text:")
print(dataset["test"][0]["plain_text"])
print("\nSummary:")
print(dataset["test"][0]["summary"])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e1a99119adee0fcb/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Split lengths: [540, 270, 90]
Features: ['plain_text', 'summary']

Plain_text:
The Service is only available to individuals aged 13 years or older Apple reserves the right at any time to modify this Agreement and to impose new or additional terms or conditions on your use of the Service, provided that Apple will give you 30 days’ advance notice of any material adverse change to the Service or applicable terms of service, unless it would not be reasonable to do so due to circumstances arising from legal, regulatory, or governmental action. Apple and its partners and licensors must collect, use, transmit, process and maintain your location data, including but not limited to the geographic location of your device and information related to your iCloud account (“Account”) and any devices registered thereunder, including but not limited to your Apple ID, device ID and name, and device type You may withdraw consent to Apple and its partners’ and licensors’ collection, use, transmission, proc

In [ ]:
#dataset.set_format(type="pandas")
#df = dataset["train"][:]

#df["WordsCount"] = df["plain_text"].str.split().apply(len)
#df["SummaryCount"] = df["summary"].str.split().apply(len)
#df.boxplot("WordsCount", grid=False, showfliers=False, color="black")


In [ ]:
#df.boxplot("SummaryCount", grid=False, showfliers=False, color="black")


In [5]:
#Select the hardware to use
device = "cuda" if torch.cuda.is_available() else "cpu"

## Model selection

In [6]:
#Model selection

model_ckpt = 'facebook/bart-large-cnn'
#model_ckpt = 't5-base'
#model_ckpt = 'google/pegasus-cnn_dailymail' #test also XSUM
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

# 

In [7]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [8]:
def evaluate_summaries(dataset, metric, model, tokenizer, batch_size=1, device=device, column_text="plain_text", column_summary="summary"):
    tos_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))
    for tos_batch, target_batch in tqdm(
        zip(tos_batches, target_batches), total=len(tos_batches)):
        inputs = tokenizer(tos_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                        attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                 clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
    
    score = metric.compute()
    return score

## Tokenization

In [9]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["plain_text"], max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length=128, truncation=True)
    return {"input_ids": input_encodings["input_ids"], "attention_mask": input_encodings["attention_mask"], 
            "labels": target_encodings["input_ids"]}


dataset_tos = dataset.map(convert_examples_to_features, batched=True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_tos.set_format(type="torch", columns=columns)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Training

In [10]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
 output_dir='training_log', num_train_epochs=2, warmup_steps=500,
 per_device_train_batch_size=1, per_device_eval_batch_size=1,
 weight_decay=0.01, logging_steps=1, push_to_hub=False,
 evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
 gradient_accumulation_steps=16)

trainer = Trainer(model=model, args=training_args,
 tokenizer=tokenizer, data_collator=seq2seq_data_collator,
 train_dataset=dataset_tos["train"],
 eval_dataset=dataset_tos["validation"])

trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: plain_text, summary. If plain_text, summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 540
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 16
  Total optimization steps = 66


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=66, training_loss=2.6610709699717434, metrics={'train_runtime': 363.8117, 'train_samples_per_second': 2.969, 'train_steps_per_second': 0.181, 'total_flos': 1304218150428672.0, 'train_loss': 2.6610709699717434, 'epoch': 1.98})

In [11]:
trainer.model.save_pretrained(f"content/model")

Configuration saved in content/model/config.json
Model weights saved in content/model/pytorch_model.bin


## Evaluation

In [12]:
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize
from datasets import load_dataset

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [20]:
score_rouge = evaluate_summaries(
    dataset["test"], rouge_metric, trainer.model, tokenizer,
    batch_size=1, column_text="plain_text", column_summary="summary")

pd.DataFrame(score_rouge, index=[f"{model_ckpt}"])

  0%|          | 0/270 [00:00<?, ?it/s]

{'input_ids': tensor([[   0,  133, 1841,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


  0%|          | 1/270 [00:04<19:22,  4.32s/it]

{'input_ids': tensor([[    0, 20770, 17005,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


  1%|          | 2/270 [00:07<16:44,  3.75s/it]

...

{'input_ids': tensor([[   0, 3972,  104,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


100%|█████████▉| 269/270 [08:32<00:01,  1.64s/it]

{'input_ids': tensor([[   0, 1106,  370,  ..., 1122,  518,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


100%|██████████| 270/270 [08:35<00:00,  1.91s/it]


,rouge1,rouge2,rougeL,rougeLsum
facebook/bart-large-cnn,0.334299,0.162122,0.221061,0.221369


In [28]:
summaries = {}
sample_text = dataset["test"][0]["plain_text"]

pipe = pipeline("summarization", model =f"content/model", tokenizer=tokenizer, device=0)
pipe_out = pipe(sample_text)
summaries["Result"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
print(summaries)

loading configuration file content/model/config.json
Model config BartConfig {
  "_name_or_path": "content/model",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  

{'Result': 'The service is only available to individuals aged 13 years or older.\nThe service may change its terms at any time without prior notice.\nYou agree to allow the service to collect, use, transmit, process and maintain your location data.\nThis service may use your personal data for marketing purposes.\nYou can opt out of location-based features by turning off Find My (including the predecessor apps Find My iPhone and Find My Friends, collectively referred to as “Find My”)'}


## Not fine tuned

In [32]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

score_rouge = evaluate_summaries(
    dataset["test"], rouge_metric, base_model, tokenizer,
    batch_size=1, column_text="plain_text", column_summary="summary")

pd.DataFrame(score_rouge, index=[f"{model_ckpt}"])

loading configuration file https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/199ab6c0f28e763098fd3ea09fd68a0928bb297d0f76b9f3375e8a1d652748f9.930264180d256e6fe8e4ba6a728dd80e969493c23d4caa0a6f943614c52d34ab
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_gen

{'input_ids': tensor([[   0,  133, 1841,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


  0%|          | 1/270 [00:02<10:43,  2.39s/it]

{'input_ids': tensor([[    0, 20770, 17005,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


  1%|          | 2/270 [00:03<08:00,  1.79s/it]

...

{'input_ids': tensor([[   0, 3972,  104,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


100%|█████████▉| 269/270 [07:11<00:01,  1.43s/it]

{'input_ids': tensor([[   0, 1106,  370,  ..., 1122,  518,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


100%|██████████| 270/270 [07:13<00:00,  1.61s/it]


,rouge1,rouge2,rougeL,rougeLsum
facebook/bart-large-cnn,0.210886,0.037177,0.126374,0.126581


In [34]:
summaries = {}
sample_text = dataset["test"][0]["plain_text"]

pipe = pipeline("summarization", model =base_model, tokenizer=tokenizer, device=0)
pipe_out = pipe(sample_text)
summaries["Result"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))
print(summaries)

{'Result': 'Apple reserves the right at any time to modify this Agreement and to impose new or additional terms or conditions on your use of the Service.\nApple will give you 30 days’ advance notice of any material adverse change to the Service or applicable terms of service.\nThe Service is only available to individuals aged 13 years or older.\nYou acknowledge and agree that Apple may, without liability to you, access, use, preserve and/or disclose your Account information and Content.'}
